**This is the TimeHigherEducation_WorldRank_Parser notebook**
* Data Cleaning: Columns are cleaned and process to prepare for analysis. 
* Data Merging: An important step is gathering information on Longitude and Latitude for the streamlit map generation. 
* City names are found from the full address using City names from cities500.txt, a file from Geonames (http://download.geonames.org/export/dump/)
* The longitude and latitude are added using the City and Country and geopy - computationally expensive ~13 minutes to run.
* Data Visualization: The world rank data is visualized using streamlit using plotly express, scatter_geo. **refer to the streamlit app**


In [ ]:
import os
import re
import pandas as pd

In [ ]:
#Parsing the World Rank Data as df_rank

In [ ]:
# read in the dataframe
df_rank = pd.read_csv("TimesHigherEducation_WorldRankings.csv")

In [ ]:
# Cleaning the dataframe
df_rank['intl_students'] = df_rank['intl_students'].replace('%', '', regex=True)
df_rank['rank'] = df_rank['rank'].replace('\–\d*|\+', '', regex=True)
df_rank['overall_score'] = df_rank['overall_score'].replace('.*\–', '', regex=True)
df_rank['number_students'] = df_rank['number_students'].replace(',', '', regex=True)

# Remove any rows without a university name and format the names
df_rank = df_rank.dropna(subset=['name'])
df_rank['name'] = df_rank['name'].replace('/world-university-rankings/', '', regex=True).str.strip()
df_rank['name'] = df_rank['name'].replace('-', ' ', regex=True).str.strip()

In [ ]:
# Load the GeoNames dataset of the city names: http://download.geonames.org/export/dump/
# cities500.zip: conatins all cities with a population > 500 
geo_df = pd.read_csv('cities500.txt', sep='\t', header=None, names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code', 'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation', 'dem', 'timezone', 'modification_date'])

# Extract these city names from the GeoNames dataset, when we are matching for the city, we do not want to match to these.
city_names = geo_df['name'].tolist()
city_names.remove('University')
city_names.remove('Box')
city_names.remove('Street')
city_names.remove('College')

In [ ]:
# First, extract the country and city from the "address" column. The country is at the end of the address
df_rank['Country'] = df_rank['address'].apply(lambda x: x.rsplit(',', 1)[-1].strip() if pd.notnull(x) else '') 

# The city is added if there is a match in the city_names list
#df_rank['City'] = df_rank['address'].apply(lambda x: next((word for word in x.split(',') if word.strip() in city_names), None) if pd.notnull(x) else None)

# Updated to find all of the matches of the address, then select only the first
df_rank['City'] = df_rank['address'].apply(lambda x: [word.strip() for word in x.split(',') if word.strip() in city_names] if pd.notnull(x) else [])

# Select the first entry in the 'City' column
df_rank['City'] = df_rank['City'].apply(lambda x: x[0] if len(x) > 0 else None)

In [ ]:
# Now geopy geocoders is used to add longitude and latitude based on the city and country, this won't always be correct

import time
from geopy.geocoders import Nominatim

city_list = df_rank['City'].tolist()
country_list = df_rank['Country'].tolist()

coordinates = [None]*len(city_list)
geolocator = Nominatim(user_agent="mathesoa") # Replace "your_app_name" with a custom user agent to avoid request throttling

chunk_size = 50
total_chunks = (len(city_list) + chunk_size - 1) // chunk_size

for chunk in range(total_chunks):
    start_index = chunk * chunk_size
    end_index = min((chunk + 1) * chunk_size, len(city_list))

    for i, (city, country) in enumerate(zip(city_list[start_index:end_index], country_list[start_index:end_index])):
        if city is not None and country is not None:
            try:
                location = geolocator.geocode(f"{city}, {country}")
                coordinates[start_index + i] = (location.latitude, location.longitude)
            except:
                pass
    
    if chunk < total_chunks - 1:
        print('first wait time')
        time.sleep(10)  # Pause for 10 seconds between chunks

In [ ]:
# Create the longitude and latitude columns in the dataframe
df_rank['Latitude'] = [coord[0] if coord is not None else None for coord in coordinates]
df_rank['Longitude'] = [coord[1] if coord is not None else None for coord in coordinates]

In [ ]:
df_rank.to_csv('World_Rank_Plot.csv', index=False)

In [ ]:
df_rank.drop('all_city_matches', axis=1, inplace=True)

In [ ]:
# Does not currently work, but in principle the address should be used to find the more accurate lon/lat values as well as the state

from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="mathesoa")

chunk_size = 50
total_chunks = (len(df_rank) + chunk_size - 1) // chunk_size

def get_state(address):
    try:
        location = geolocator.geocode(address)
        state = location.raw['address']['state']
        return state
    except:
        return ''

df_rank['State'] = df_rank['address'].apply(get_state)

for chunk in range(total_chunks):
    start_index = chunk * chunk_size
    end_index = min((chunk + 1) * chunk_size, len(df_rank))

    # Get chunk of data
    df_chunk = df_rank.loc[start_index:end_index].copy() # create a copy

    # apply the function to each row in the chunk
    df_chunk['State'] = df_chunk.apply(lambda row: get_state(row['address']), axis=1)

    # Update dataframe with results
    df_rank.loc[start_index:end_index] = df_chunk
    
    # Pause between chunks
    if chunk < total_chunks - 1:
        print('first wait time')
        time.sleep(10)